In [ ]:
# konlpy Mecab 사용하기

!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

In [ ]:
!pip install glove_python

     |████████████████████████████████| 266kB 7.5MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=702587 sha256=51f94269367a23274ebb26f9c7507443dc93ec63dc82b52cfc9f829956a3accf
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [2]:
# 내 드라이브에 대한 주소
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
path = "/gdrive/My Drive/dacon_news"

train = pd.read_csv(path + "/data/news_train.csv")
test = pd.read_csv(path + "/data/news_test.csv")

# word 전처리 및 형태소 분리

### 특수문자를 포함하지 않은 전처리

In [ ]:
from konlpy.tag import Mecab
from konlpy.tag import Okt
import re

def text_preprocessing(text_list):
    
    stopwords = ['이'] #불용어 설정
    
    tokenizer = Mecab() #형태소 분석기 
    token_list = [] 
    
    for text in text_list:
        txt = re.sub('[^가-힣]', ' ', text) #한글만 남기고 다른 글자 모두 제거
        txt = re.sub('[가-힣\s]+기자]','기자', txt) #기자 이름 제거
        token = tokenizer.morphs(txt) #형태소 분석

        #형태소 분석 결과 중 stopwords에 해당하지 않고, float type이 아닌 것만 수집
        token = [t for t in token if t not in stopwords] 
        token_list.append(token)
        
    return token_list, tokenizer

#형태소 분석기를 따로 저장한 이유는 후에 test 데이터 전처리를 진행할 때 이용해야 되기 때문입니다. 
train['new_article'], okt = text_preprocessing(train['content'])
test['new_article'], okt = text_preprocessing(test['content'])

### 특수문자를 포함한 전처리

In [63]:
from konlpy.tag import Mecab
import re
from konlpy.tag import Okt

def text_preprocessing(text_list):
    
    stopwords = ['이']
    tokenizer = Mecab() #형태소 분석기

    token_list = [] 
    
    for text in text_list:
        txt = re.sub('[a-zA-Z0-9]', ' ', text) #영문 제거
        txt = re.sub('[가-힣\s]+기자]','기자', txt) #기자 이름 제거
        token = tokenizer.morphs(txt) #형태소 분석

        token = [t for t in token if t not in stopwords] 
        token_list.append(token)
        
    return token_list, tokenizer

#형태소 분석기를 따로 저장한 이유는 후에 test 데이터 전처리를 진행할 때 이용해야 되기 때문입니다.
embedding_train = pd.concat([train["content"],test["content"]],axis=0)
embedding_train, okt = text_preprocessing(embedding_train)

# Word2Vec 학습시키기

In [6]:
from gensim.models.word2vec import Word2Vec

In [67]:
embedding_model = Word2Vec(embedding_train,         # 리스트 형태의 데이터
                 sg=1,          # 0: CBOW, 1: Skip-gram
                 size = 300,      # 벡터 크기(300)
                 window = 10,     # 고려할 앞뒤 폭(앞뒤 6단어)
                 min_count = 30,  # 사용할 단어의 최소 빈도(30회 이하 단어 무시)
                 workers = -1)    # 동시에 처리할 작업 수(코어 수와 비슷하게 설정)

In [68]:
embedding_model.save("embedding_300_10_30_all.model")

In [70]:
embedding_model.most_similar("종목")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('새로', 0.2245328724384308),
 ('뛴', 0.20319154858589172),
 ('성장세', 0.20244765281677246),
 ('닷새', 0.19612061977386475),
 ('야기', 0.1954067051410675),
 ('식사', 0.17853125929832458),
 ('임대', 0.1767268031835556),
 ('든지', 0.1745581179857254),
 ('중립', 0.16294974088668823),
 ('독감', 0.16176140308380127)]

In [56]:
embedding_model.most_similar('카톡')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('방사광', 0.2288285493850708),
 ('폰', 0.21296778321266174),
 ('대원', 0.210852712392807),
 ('노리', 0.20738786458969116),
 ('분양', 0.20020851492881775),
 ('봉', 0.19835519790649414),
 ('형사', 0.19768759608268738),
 ('마켓', 0.19744479656219482),
 ('기원', 0.19487667083740234),
 ('푸르', 0.19428278505802155)]